# Extracting resonance data from an ENDF file
## Overview

Resonances are stored in an ENDF file in MF2 ('file 2') MT151 ('section 151'). The hierarchy of MF2 is as follows:

```
-- Section 151
    -- Isotope 0
        -- Energy range 0
        -- Energy range 1 
           ...
    -- Isotope 1
       ...
```

In modern files there is only one isotope per file, but the level still exists.

## Reich-Moore (LRF = 3)

In [1]:
import ENDFtk

Pb208_tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pb208.endf' )
Pb208_mat = Pb208_tape.materials.front()

Pb208_file2 = Pb208_mat.file(2).parse()
Pb208_section151 = Pb208_file2.section(151)
Pb208_resonance_region = Pb208_section151.isotopes[0]

Pb208_resonance_region

The whole resonance region for Pb208 is contained in the `ENDFtk.MF2.MT151.Isotope` object. Documentation for all of these classes can be found by running `help(object)`

In [2]:
help(Pb208_resonance_region)

Help on Isotope in module ENDFtk.MF2.MT151 object:

class Isotope(pybind11_builtins.pybind11_object)
 |  MF2 MT151 section - an isotope with resonance energy ranges
 |
 |  Method resolution order:
 |      Isotope
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |
 |      1. __init__(self: ENDFtk.MF2.MT151.Isotope, zai: int, abn: float, lfw: bool, ranges: list[ENDFtk.MF2.MT151.ResonanceRange]) -> None
 |
 |      Initialise the component
 |
 |      Arguments:
 |          self      the component
 |          zai       the za of the isotope
 |          abn       the abundance of the isotope
 |          lfw       the lfw flag for unresolved resonances
 |          ranges    the resonance ranges defined for the isotope
 |
 |      2. __init__(self: ENDFtk.MF2.MT151.Isotope, component: ENDFtk.MF2.MT151.Isotope) -> None
 |
 |      Copy the component
 |
 |      Argumen

The RRR is the first energy range, and the URR (if there is one) is typically the second. In this file, there is no URR so the number of energy ranges is 1:

In [3]:
Pb208_resonance_region.number_resonance_ranges

1

In [4]:
Pb208_RRR = Pb208_resonance_region.resonance_ranges[0]
Pb208_RRR

In this case, the type is resolved (`LRU` = 1) and the representation is Reich-Moore (`LRF` = 3)


In [5]:
print(f"Resonance range limits: {Pb208_RRR.lower_energy:.1E} - {Pb208_RRR.upper_energy:.1E} eV")
print(f"Type (LRU): {Pb208_RRR.type}")
print(f"Representation (LRF): {Pb208_RRR.representation}")
print(f"Energy-dependent scattering radius: {Pb208_RRR.energy_dependent_scattering_radius}")
print(f"Scattering radius calculation method (NAPS): {Pb208_RRR.scattering_radius_calculation_option}")

Resonance range limits: 1.0E-05 - 1.5E+06 eV
Type (LRU): 1
Representation (LRF): 3
Energy-dependent scattering radius: False
Scattering radius calculation method (NAPS): 1


The parameters are in the `parameters` attribute, which is a `ENDFtk.MF2.MT151.ReichMoore` object:

In [6]:
Pb208_parameters = Pb208_RRR.parameters
Pb208_parameters

In [7]:
print(f"The RRR representation (LRF): {Pb208_parameters.representation}")
print(f"Type (LRU): {Pb208_parameters.type}")
print(f"The target spin (SPI): {Pb208_parameters.spin}")
print(f"The scattering radius (AP): {Pb208_parameters.scattering_radius} fm")
print(f"Angular distributions can be used to calculate angular distributions (LAD): {Pb208_parameters.angular_distributions_flag}")
print(f"Number of l-values for angular distribution convergence (NLSC): {Pb208_parameters.number_l_values_for_distributions}")
print(f"The number of l-values with parameters (NLS): {Pb208_parameters.number_l_values}")

The RRR representation (LRF): 3
Type (LRU): 1
The target spin (SPI): 0.0
The scattering radius (AP): 0.975 fm
Angular distributions can be used to calculate angular distributions (LAD): True
Number of l-values for angular distribution convergence (NLSC): 4
The number of l-values with parameters (NLS): 4


For LRF=3, the parameters are stored by l-value in the `l_values` attribute, which is a list of `NLS` `ReichMooreLValue` objects:

In [8]:
Pb208_parameters.l_values[:]

In [9]:
Pb208_s_wave = Pb208_parameters.l_values[0]
Pb208_s_wave

In [10]:
print(f"AWR for this isotope: {Pb208_s_wave.atomic_weight_ratio}")
print(f"l value for this l-group (L): {Pb208_s_wave.orbital_momentum}")
print(f"Number of resonances in this l-group (NRS): {Pb208_s_wave.number_resonances} ")
print(f"The radius for this l-group (APL): {Pb208_s_wave.l_dependent_scattering_radius}")

AWR for this isotope: 206.19
l value for this l-group (L): 0
Number of resonances in this l-group (NRS): 9 
The radius for this l-group (APL): 0.967


The resonance energies, J values, and widths are stored in lists that are `NRS` long:

In [11]:
print(f"Energy [eV] \t J \t Gamma_n \t Gamma_g \t Gamma_f1 \t Gamma_f2")
print("-"*82)
for Er, J, n, g, f1, f2 in zip(Pb208_s_wave.resonance_energies,
                               Pb208_s_wave.spin_values, 
                               Pb208_s_wave.neutron_widths, 
                               Pb208_s_wave.gamma_widths, 
                               Pb208_s_wave.first_fission_widths, 
                               Pb208_s_wave.second_fission_widths):
    print(f"{Er:.2E} \t {J} \t {n} \t {g}  \t {f1}   \t         {f2}")

Energy [eV] 	 J 	 Gamma_n 	 Gamma_g 	 Gamma_f1 	 Gamma_f2
----------------------------------------------------------------------------------
-1.91E+06 	 0.5 	 273070.0 	 0.0914  	 0.0   	         0.0
5.06E+05 	 0.5 	 53590.0 	 0.166  	 0.0   	         0.0
8.88E+05 	 0.5 	 5490.0 	 0.166  	 0.0   	         0.0
9.98E+05 	 0.5 	 2653.99 	 0.166  	 0.0   	         0.0
1.07E+06 	 0.5 	 4322.564 	 0.166  	 0.0   	         0.0
1.11E+06 	 0.5 	 2680.318 	 0.166  	 0.0   	         0.0
1.24E+06 	 0.5 	 1157.726 	 0.00186  	 0.0   	         0.0
1.36E+06 	 0.5 	 616.568 	 0.00186  	 0.0   	         0.0
8.00E+06 	 0.5 	 445770.0 	 0.166  	 0.0   	         0.0


The lists can be accessed in Python several ways:

In [ ]:
Pb208_s_wave.resonance_energies[:]

In [ ]:
Pb208_s_wave.resonance_energies.to_list()

In [ ]:
for Er in Pb208_s_wave.resonance_energies:
    print(Er)

## R-Matrix Limited (LRF = 7)

In [ ]:
Rh103_tape = ENDFtk.tree.Tape.from_file( 'resources/n-Rh103.endf' )
Rh103_mat = Rh103_tape.materials.front()

Rh103_file2 = Rh103_mat.file(2).parse()
Rh103_section151 = Rh103_file2.section(151)
Rh103_resonance_region = Rh103_section151.isotopes[0]

Rh103_resonance_region

This evaluation has both RRR and URR sections:

In [ ]:
Rh103_resonance_region.number_resonance_ranges

In [ ]:
Rh103_RRR = Rh103_resonance_region.resonance_ranges[0]

print(f"Resonance range limits: {Rh103_RRR.lower_energy:.1E} - {Rh103_RRR.upper_energy:.1E} eV")
print(f"Type (LRU): {Rh103_RRR.type}")
print(f"Representation (LRF): {Rh103_RRR.representation}")
print(f"Energy-dependent scattering radius: {Rh103_RRR.energy_dependent_scattering_radius}")
print(f"Scattering radius calculation method (NAPS): {Rh103_RRR.scattering_radius_calculation_option}")

The parameters are in the `parameters` attribute, which is a `ENDFtk.MF2.MT151.RMatrixLimited` object

In [ ]:
Rh103_parameters = Rh103_RRR.parameters
Rh103_parameters

In [ ]:
print(f"The RRR representation (KRM): {Rh103_parameters.representation}")
print(f"The range type (LRU): {Rh103_parameters.type}")
print(f"The target spin (SPI): {Rh103_parameters.spin}")
print(f"Relativistic kinematics should be used (KRL): {Rh103_parameters.non_relativistic_kinematics}")
print(f"The widths are in units [eV^1/2] instead of [eV] (IFG): {Rh103_parameters.reduced_widths}")
print(f"The scattering radius (AP): {Rh103_parameters.scattering_radius}")
print(f"The number of spin groups (NJS): {Rh103_parameters.number_spin_groups}")

The particle-pair information is in the `particle_pairs` attribute:

In [ ]:
Rh103_pp = Rh103_parameters.particle_pairs
Rh103_pp

In [ ]:
print(f"Number of particle-pairs: {Rh103_pp.number_particle_pairs}")

In [ ]:
for MT, ZA, ZB, IA, IB, MA, MB, PA, PB, PNT, Q, SHF in zip(Rh103_pp.MT, 
                                                          Rh103_pp.charge_particle_A,
                                                          Rh103_pp.charge_particle_B,
                                                          Rh103_pp.spin_particle_A,
                                                          Rh103_pp.spin_particle_B,
                                                          Rh103_pp.mass_particle_A,
                                                          Rh103_pp.mass_particle_B,
                                                          Rh103_pp.parity_particle_A,
                                                          Rh103_pp.parity_particle_B,
                                                          Rh103_pp.penetrability_flag,
                                                          Rh103_pp.Q,
                                                          Rh103_pp.shift_factor_flag
                                                          ):
    print(f"Particle pair representing MT{MT}:")
    print(f"  Particle A")
    print(f"     Charge (ZA): {ZA}")
    print(f"     Mass (MA): {MA}")
    print(f"     Spin (IA): {IA}")
    print(f"     Parity (PA): {PA}")
    print(f"  Particle B")
    print(f"     Charge (ZB): {ZB}")
    print(f"     Mass (MB): {MB}")
    print(f"     Spin (IB): {IB}")
    print(f"     Parity (PB): {PB}")
    print(f"  Q-value: {Q}")
    print(f"  Penetrability calculation flag (PNT): {PNT}")
    print(f"  Shift factor flag (SHF): {SHF}")

    print()

The parameters are split by spin group (`J`) in RML

In [ ]:
Rh103_first_spin_group = Rh103_parameters.spin_groups[0]
Rh103_first_spin_group

In [ ]:
print(f"Spin group J-value (AJ): {Rh103_first_spin_group.spin}")
print(f"Spin group parity (PJ): {Rh103_first_spin_group.parity}")
print(f"Number of channels in this spin group (NCH): {Rh103_first_spin_group.number_channels}")
print(f"Number of channels with background values (KBK): {Rh103_first_spin_group.number_background_channels}")
print(f"Number of resonances in this spin group (NRS): {Rh103_first_spin_group.number_resonances}")

The parameters are in the `parameters` attribute:

In [ ]:
Rh103_first_spin_group.parameters

In [ ]:
print(f"Number of resonances in this group (NRS): {Rh103_first_spin_group.parameters.number_resonances}")

In [ ]:
print(f"Energy [eV] \t 1st channel\t 2nd  \t\t3rd\t 4th\t 5th ")
print("-"*82)
for er, res in zip(Rh103_first_spin_group.parameters.resonance_energies,Rh103_first_spin_group.parameters.resonance_parameters):
    print(er, end=' \t  ')
    for width in res:
        print(f"{width}",end='\t')
    print()

## The URR

The URR allows for only one RMatrix formalism, but has two potential representations (`LRF` flag)

In [ ]:
Rh103_URR = Rh103_resonance_region.resonance_ranges[1]
Rh103_URR

In [ ]:
print(f"Resonance range type (LRU): {Rh103_URR.type}")
print(f"URR energy limits: {Rh103_URR.lower_energy:.1E} - {Rh103_URR.upper_energy:.1E} eV")
print(f"Scattering radius flag (NAPS): {Rh103_URR.scattering_radius_calculation_option}")
print(f"Scattering radius is energy-dependent (NRO): {Rh103_URR.energy_dependent_scattering_radius}")
print(f"An average fission widths are given (LFW): {Rh103_URR.average_fission_width_flag}")
print(f"Parameter energy-dependence flag (LRF): {Rh103_URR.representation}")

The parameters are in the `parameters` attribute:

In [ ]:
Rh103_URR_parameters = Rh103_URR.parameters
Rh103_URR_parameters

In [ ]:
print(f"The target spin (SPI): {Rh103_URR_parameters.spin}")
print(f"The scattering radius (AP): {Rh103_URR_parameters.scattering_radius}")
print(f"Average fission widths are given (LFW): {Rh103_URR_parameters.average_fission_width_flag}")
print(f"These parameters should only be used for self-shielding calculations (LSSF): {Rh103_URR_parameters.self_shielding_only}")
print(f"Number of l-groups (NLS): {Rh103_URR_parameters.number_l_values}")

The parameters are within each l-group:

In [ ]:
Rh103_s_wave = Rh103_URR_parameters.l_values[0]
Rh103_s_wave

In [ ]:
print(f"L value for this group: {Rh103_s_wave.orbital_momentum}")
print(f"Atomic weight ratio for this isotope (AWRI): {Rh103_s_wave.atomic_weight_ratio}")
print(f"Number of J values in this groups (NJS): {Rh103_s_wave.NJS}")


The parameters are then categorized by `J` group:

In [ ]:
Rh103_URR_first_spin_group = Rh103_s_wave.j_values[0]
help(Rh103_URR_first_spin_group)

In [ ]:
print(f"J-value for this group (AJ): {Rh103_URR_first_spin_group.spin}")
print(f"Degrees of freedom for the competitive channel (AMUX): {Rh103_URR_first_spin_group.competitive_width_degrees_freedom}")
print(f"Degrees of freedom for the neutron channel (AMUN): {Rh103_URR_first_spin_group.neutron_width_degrees_freedom}")
print(f"Degrees of freedom for the gamma channel (AMUG): {Rh103_URR_first_spin_group.gamma_width_degrees_freedom}")
print(f"Degrees of freedom for the fission channel (AMUF): {Rh103_URR_first_spin_group.fission_width_degrees_freedom}")

In [ ]:
print("Energy [eV]\t  <D> \t\t Gn0\t\tGg\t\tGf\tGx")
print("-"*75)
for energy, level_spacing, gn0, gg, gf, gx in zip(
    Rh103_URR_first_spin_group.energies,
    Rh103_URR_first_spin_group.average_level_spacings,
    Rh103_URR_first_spin_group.average_neutron_widths,
    Rh103_URR_first_spin_group.average_gamma_widths,
    Rh103_URR_first_spin_group.average_fission_widths,
    Rh103_URR_first_spin_group.average_competitive_widths
):
    print(f"{energy:.2E}\t{level_spacing} \t{gn0}\t{gg}\t{gf}\t{gx}")